In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q espnet==0.9.5 parallel_wavegan==0.4.8

     |████████████████████████████████| 716kB 6.1MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 419kB 51.9MB/s 
     |████████████████████████████████| 1.0MB 47.3MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 1.4MB 51.9MB/s 
     |████████████████████████████████| 276kB 53.6MB/s 
     |████████████████████████████████| 13.1MB 248kB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 245kB 50.7MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
     |████████████████████████████████| 317kB 54.8MB/s 
     |████████████████████████████████| 225kB 47.4MB/s 
     |████████████████████████████████| 174kB 51.3MB/s 
     |████████████████████████████████| 1.7MB 22.1MB/s 
     |████████████████████████████████| 184kB 51.6MB/s 
     |████████████████████████████████| 1.3MB 49.4MB/s 
 

In [ ]:
###################################
#          ENGLISH MODELS         #
###################################
fs, lang = 24000, "English"
tag = "kan-bayashi/vctk_gst_tacotron2"
# tag = "kan-bayashi/vctk_gst_transformer"
vocoder_tag = "vctk_parallel_wavegan.v1"
# vocoder_tag = "vctk_multi_band_melgan.v2"

In [ ]:
import time
import torch
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.tts_inference import Text2Speech
from parallel_wavegan.utils import download_pretrained_model
from parallel_wavegan.utils import load_model
d = ModelDownloader()
text2speech = Text2Speech(
    **d.download_and_unpack(tag),
    device="cuda",
    # Only for Tacotron 2
    threshold=0.5,
    minlenratio=0.0,
    maxlenratio=10.0,
    use_att_constraint=False,
    backward_window=1,
    forward_window=3,
    # Only for FastSpeech & FastSpeech2
    speed_control_alpha=1.0,
)
text2speech.spc2wav = None  # Disable griffin-lim
# NOTE: Sometimes download is failed due to "Permission denied". That is 
#   the limitation of google drive. Please retry after serveral hours.
vocoder = load_model(download_pretrained_model(vocoder_tag)).to("cuda").eval()
vocoder.remove_weight_norm()

In [ ]:
# decide the input sentence by yourself
print(f"Input your favorite sentence in {lang}.")
x = input()

# you can change here to load your own reference speech
# e.g.
import soundfile as sf
speech, fs = sf.read("/content/ankit.wav") #you can upload any voice spoken in English for reference
speech = torch.from_numpy(speech).float()
speech = torch.randn(50000,)

# synthesis
with torch.no_grad():
    start = time.time()
    wav, c, *_ = text2speech(x, speech=speech)
    wav = vocoder.inference(c)
rtf = (time.time() - start) / (len(wav) / fs)
print(f"RTF = {rtf:5f}")

# let us listen to generated samples
from IPython.display import display, Audio
display(Audio(wav.view(-1).cpu().numpy(), rate=fs))

Input your favorite sentence in English.
hello my life is going fine


/usr/local/lib/python3.6/dist-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/usr/local/lib/python3.6/dist-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  normalized, onesided, return_complex)


RTF = 0.567480
